# 1.Import libraries and dataset

In [ ]:
# import libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from sklearn.datasets import fetch_20newsgroups

# 2. Preprocessing

In [ ]:

data= fetch_20newsgroups(subset='all',  shuffle=True, random_state=46)
data_train = fetch_20newsgroups(
    subset="train",  shuffle=True, random_state=46
)
data_test = fetch_20newsgroups(
    subset="test",
    shuffle=True,
    random_state=46,
)
print(data.target_names)
print(data.target_names[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
alt.atheism


In [ ]:
print(data_train.keys()) # check attribute

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [ ]:
#  依照 data.target 中的每個數字(共計 18846 筆)，找到 data.target_name (僅有20種)所對應的文字
import pandas as pd
# data_category=[]
# for i in data.target:
#     data_category.append(data.target_names[i])
# print(data_category[:5])

data_train_category = []
for i in data_train.target:
    data_train_category.append(data_train.target_names[i])
print(data_train_category[:5])


data_test_category=[]
for i in data_test.target:
    data_test_category.append(data_test.target_names[i])
print(data_test_category[:5])

['rec.autos', 'sci.space', 'sci.electronics', 'comp.sys.mac.hardware', 'talk.politics.mideast']
['talk.politics.mideast', 'talk.politics.mideast', 'comp.graphics', 'alt.atheism', 'talk.politics.mideast']


In [ ]:
# turn it into a dataframe

# data_df=pd.DataFrame(data.target,columns=['Category_No'])

data_train_df = pd.DataFrame(data_train.target, columns=["Category_No"])
data_test_df = pd.DataFrame(data_test.target, columns=["Category_No"])


# data_df['Category_No'].value_counts()    # 了解每個類別的數量
# data_df['Text'] = data.data    # 將文字放入dataframe
data_train_df['Text']=data_train.data
data_test_df['Text'] = data_test.data


# data_df['Category_Name']=data_category  #將類別名稱放入dataframe

data_train_df["Category_Name"] = data_train_category
data_test_df["Category_Name"] = data_test_category

data_train_df.head(5)
data_test_df.head(5)


# data_df.head(5)

# data_df=data_df[:5]
# data_df

,Category_No,Text,Category_Name
0,17,From: benali@alcor.concordia.ca ( ILYESS B. BD...,talk.politics.mideast
1,17,From: sera@zuma.UUCP (Serdar Argic)\nSubject: ...,talk.politics.mideast
2,1,From: gord@jericho.uucp (Gord Wait S-MOS Syste...,comp.graphics
3,0,From: jaeger@buphy.bu.edu (Gregg Jaeger)\nSubj...,alt.atheism
4,17,From: jake@bony1.bony.com (Jake Livni)\nSubjec...,talk.politics.mideast


In [ ]:
pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=c0d085e9a474e80eebd71a1ecd1969ecf218411544c777da79195c971be423b1
  Stored in directory: /root/.cache/pip/wheels/d9/31/ff/ff51141a088571a9f672449e5aad5ea8bb35ca5d95ba135f30
Successfully built swifter


In [ ]:
# import necessary libraries
import pandas as pd
import spacy
import swifter

# download the spacy model
nlp = spacy.load("en_core_web_sm")



def token_lemma_nonstop(text):    #  僅能丟入參數 是series 中的單筆資料
    doc = nlp(text)
    output = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(output)

def clean_and_normalize(series):
    output = series.str.lower()
    output = output.str.replace(r"[^\w\s]", " ", regex=True)
    output=output.str.replace(r"\s+", " ", regex=True)



    # 步驟 2: 使用 swifter 並行處理 NLP 任務
    output = output.swifter.apply(token_lemma_nonstop)
    return output




In [ ]:
data_train_df["Text_Clean"] = clean_and_normalize(data_train_df["Text"])
data_train_df.head(2)

Pandas Apply:   0%|          | 0/11314 [00:00<?, ?it/s]

,Category_No,Text,Category_Name,Text_Clean
0,7,From: welty@cabot.balltown.cma.COM (richard we...,rec.autos,welty cabot balltown cma com richard welty sub...
1,14,From: pyron@skndiv.dseg.ti.com (Dillon Pyron)\...,sci.space,pyron skndiv dseg ti com dillon pyron subject ...


In [ ]:
data_test_df["Text_Clean"] = clean_and_normalize(data_test_df["Text"])
data_test_df.head(2)

Pandas Apply:   0%|          | 0/7532 [00:00<?, ?it/s]

,Category_No,Text,Category_Name,Text_Clean
0,17,From: benali@alcor.concordia.ca ( ILYESS B. BD...,talk.politics.mideast,benali alcor concordia ilyess b bdira subject ...
1,17,From: sera@zuma.UUCP (Serdar Argic)\nSubject: ...,talk.politics.mideast,sera zuma uucp serdar argic subject traditiona...


### 2.1 TF-IDF Vectorizer

In [ ]:
# tfidf vectorizer code
from sklearn.feature_extraction.text import TfidfVectorizer

# modify min_df parameter to improve accuracy

tv = TfidfVectorizer(stop_words="english", ngram_range=(1, 2), min_df=1, max_df=0.75)
# tv = TfidfVectorizer(stop_words="english", ngram_range=(1, 2), min_df=5, max_df=0.75)

X_train_tf= tv.fit_transform(data_train_df.Text_Clean)
X_test_tf=tv.transform(data_test_df.Text_Clean)

In [ ]:
# view the target / outpupt
# y = data_df.Category_No
y_train = data_train_df.Category_Name
y_test = data_test_df.Category_Name

# 3 Model comparision

### 3.1 Linear model: Logistic Regression with TF-IDF Vectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# model
model_lr = LogisticRegression()
model_lr.fit(X_train_tf, y_train)

# predict
y_pred_lr = model_lr.predict(X_test_tf)

# evaluate
print(classification_report(y_test, y_pred_lr))
print("Accuracy:", accuracy_score(y_test, y_pred_lr))

                          precision    recall  f1-score   support

             alt.atheism       0.81      0.76      0.78       319
           comp.graphics       0.69      0.78      0.73       389
 comp.os.ms-windows.misc       0.75      0.76      0.76       394
comp.sys.ibm.pc.hardware       0.70      0.73      0.71       392
   comp.sys.mac.hardware       0.80      0.83      0.82       385
          comp.windows.x       0.85      0.75      0.80       395
            misc.forsale       0.74      0.84      0.79       390
               rec.autos       0.90      0.88      0.89       396
         rec.motorcycles       0.94      0.95      0.95       398
      rec.sport.baseball       0.88      0.92      0.90       397
        rec.sport.hockey       0.92      0.95      0.94       399
               sci.crypt       0.94      0.92      0.93       396
         sci.electronics       0.75      0.74      0.74       393
                 sci.med       0.90      0.83      0.86       396
         

### 3.2 Linear SVC model with TF-IDF Vectorizer

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model_svc = LinearSVC()
model_svc.fit(X_train_tf, y_train)

# predict
y_pred_svc = model_svc.predict(X_test_tf)

# evaluate
print(classification_report(y_test, y_pred_svc))
print("Accuracy:", accuracy_score(y_test, y_pred_svc))

                          precision    recall  f1-score   support

             alt.atheism       0.83      0.79      0.81       319
           comp.graphics       0.75      0.80      0.78       389
 comp.os.ms-windows.misc       0.79      0.75      0.77       394
comp.sys.ibm.pc.hardware       0.72      0.76      0.74       392
   comp.sys.mac.hardware       0.81      0.86      0.84       385
          comp.windows.x       0.88      0.77      0.82       395
            misc.forsale       0.80      0.89      0.84       390
               rec.autos       0.93      0.91      0.92       396
         rec.motorcycles       0.96      0.96      0.96       398
      rec.sport.baseball       0.91      0.94      0.92       397
        rec.sport.hockey       0.95      0.98      0.97       399
               sci.crypt       0.93      0.95      0.94       396
         sci.electronics       0.81      0.77      0.79       393
                 sci.med       0.91      0.86      0.88       396
         

## 3.3 Deep Learning model: TextCNN

In [ ]:
!pip install torchtext -q